In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import os
import time
import requests
import csv
import pandas as pd
import matplotlib.pyplot as plt
import nltk

pd.set_option('display.max_colwidth', None)

In [2]:
# Config

# Create Backup
# --------------------

os.system("mkdir backup")

256

In [ ]:
# Functions
#    []   scrapRichemont()                     #    []   scrapRichemont()
#    []   scrapDassaultAviation()              #    []   scrapRichemont()
#    []   scrapAirFrance()                     #    []   scrapRichemont()
#    []   scrapSanofi()                        #    []   scrapRichemont()
#    []   scrapHermes()                        #    []   scrapRichemont()
#    []   scrapFramatome()                     #    []   scrapRichemont()

In [3]:
# Scrap Richemont
def scrapRichemont():
    richemont_job_db = []
    r = requests.get('https://jobs.richemont.com/search/?q=&sortColumn=referencedate&sortDirection=desc&optionsFacetsDD_country=FR&startrow=1')
    soup = BeautifulSoup(r.content.decode("utf-8"),'lxml')

    for x in soup.find_all('tr', attrs={'class' : 'data-row clickable'}):
        row_scrap = []
        job_title = x.find('span', attrs={'class' : 'jobTitle'}).text.replace('\n', '')
        job_link = x.find('span', attrs={'class' : 'jobTitle'}).find('a')['href']
        company = x.find('span', attrs={'class' : 'jobFacility'}).text.replace('\n', '')
        departement = x.find('span', attrs={'class' : 'jobDepartment'}).text.replace('\n', '')
        location = x.find('span', attrs={'class' : 'jobLocation'}).text.replace('\n', '').strip()
        row_scrap.append(job_title)
        row_scrap.append(job_link)
        row_scrap.append(company)
        row_scrap.append(departement)
        row_scrap.append(location)
        richemont_job_db.append(row_scrap)

    i = 14
    while i <= int(soup.find('a', class_='paginationItemLast')['href'][-3:]) :
        r = requests.get(f'https://jobs.richemont.com/search/?q=&sortColumn=referencedate&sortDirection=desc&optionsFacetsDD_country=FR&startrow={i}')

        soup = BeautifulSoup(r.content.decode("utf-8"),'lxml')

        for x in soup.find_all('tr', attrs={'class' : 'data-row clickable'}):
            row_scrap = []
            job_title = x.find('span', attrs={'class' : 'jobTitle'}).text.replace('\n', '')
            job_link = x.find('span', attrs={'class' : 'jobTitle'}).find('a')['href']
            company = x.find('span', attrs={'class' : 'jobFacility'}).text.replace('\n', '')
            departement = x.find('span', attrs={'class' : 'jobDepartment'}).text.replace('\n', '')
            location = x.find('span', attrs={'class' : 'jobLocation'}).text.replace('\n', '').strip()
            row_scrap.append(job_title)
            row_scrap.append(job_link)
            row_scrap.append(company)
            row_scrap.append(departement)
            row_scrap.append(location)
            richemont_job_db.append(row_scrap)
        i = i + 14

    richemont_job_db = pd.DataFrame(richemont_job_db)
    richemont_job_db = richemont_job_db.rename(columns={0 : "Position", 1 : "link", 2 : "Company",
                                                        3 : "Department", 4 : "Location"})
    richemont_job_db['desc'] = ''

    # Add Desc to job offer
    # add https://jobs.richemont.com before links to connect

    i = 0

    while i <= len(richemont_job_db)-1:
        r = requests.get('https://jobs.richemont.com' + richemont_job_db['link'][i])
        soup = BeautifulSoup(r.content,'lxml')
        richemont_job_db['desc'][i] = soup.find('span', class_="jobdescription").text
        i = i + 1
    #df = pd.concat([df, richemont_job_db], axis=0)
    print("Richemont Scrap Finished")
    
#Scrap Dassault Aviation
def scrapDassaultAviation():
    dassaultAviation_db = []

    i = 1
    while i <= 10:
        r = requests.get(f'https://carriere.dassault-aviation.com/offre-de-emploi/liste-offres.aspx?page={i}&LCID=1036')
        soup = BeautifulSoup(r.content,'lxml')

        for x in soup.find_all('li', class_='ts-offer-list-item'):
                row_scrap = []
                job_title = x.find('a', attrs={'class' : 'ts-offer-list-item__title-link'}).text.replace('\n', '').strip()
                job_link = x.find('a', attrs={'class' : 'ts-offer-list-item__title-link'})['href']
                company = "Dassault Aviation"
                departement = 'N/A'
                details = x.find('ul', attrs={'class' : 'ts-offer-list-item__description'}).find_all('li')

                row_scrap.append(job_title)
                row_scrap.append(job_link)
                row_scrap.append(company)
                row_scrap.append(departement)
                row_scrap.append(details)

                dassaultAviation_db.append(row_scrap)

        i = i + 1

    dassaultAviation_db = pd.DataFrame(dassaultAviation_db)
    dassaultAviation_db = dassaultAviation_db.rename(columns={0 : "Position", 1 : "link", 2 : "Company", 3 : "Department", 4 : "Location"}) #
    dassaultAviation_db['desc'] = ''

    # Add Desc to job offer
    # add https://carriere.dassault-aviation.com before links to connect

    i = 0
    while i <= len(dassaultAviation_db)-1:
        r = requests.get('https://carriere.dassault-aviation.com' + dassaultAviation_db['link'][i])
        soup = BeautifulSoup(r.content,'lxml')
        dassaultAviation_db['desc'][i] = soup.find('div', attrs={'id' : 'contenu-ficheoffre'}).text
        i = i + 1
    #df = pd.concat([df, dassaultAviation_db], axis=0)
    print("Dassault Aviation Scrap Finished")
    
# Scrap Air France
def scrapAirFrance():
    airFrance_db = []

    r = requests.get('https://recrutement.airfrance.com/offre-de-emploi/liste-offres.aspx')
    soup = BeautifulSoup(r.content,'lxml')

    for x in soup.find_all('li', class_='ts-offer-list-item'):
                row_scrap = []
                job_title = x.find('a', attrs={'class' : 'ts-offer-list-item__title-link'}).text.replace('\n', '').strip()
                job_link = x.find('a', attrs={'class' : 'ts-offer-list-item__title-link'})['href']
                company = "Air France"
                departement = 'N/A'
                details = x.find('ul', attrs={'class' : 'ts-offer-list-item__description'}).find_all('li')

                row_scrap.append(job_title)
                row_scrap.append(job_link)
                row_scrap.append(company)
                row_scrap.append(departement)
                row_scrap.append(details)

                airFrance_db.append(row_scrap)

    airFrance_db = pd.DataFrame(airFrance_db)
    airFrance_db = airFrance_db.rename(columns={0 : "Position", 1 : "link", 2 : "Company", 3 : "Department", 4 : "Location"}) #
    airFrance_db['desc'] = ''

    i = 0
    while i <= len(airFrance_db)-1:
        r = requests.get('https://recrutement.airfrance.com' + airFrance_db['link'][i])
        soup = BeautifulSoup(r.content,'lxml')
        airFrance_db['desc'][i] = soup.find('div', attrs={'id' : 'contenu-ficheoffre'}).text
        i = i + 1
    #df = pd.concat([df, airFrance_db], axis=0)
    print("Air France Scrap Finished")

# Scrap Sanofi
def scrapSanofi():
    r = requests.get('https://fr.jobs.sanofi.com/recherche-d%27offres/?p=1')
    soup = BeautifulSoup(r.content,'lxml')

    sanofi_db = []

    i = 1
    while i <= int(soup.find('input', class_='pagination-current')['max']):
        r = requests.get(f'https://fr.jobs.sanofi.com/recherche-d%27offres/?p={i}')
        soup = BeautifulSoup(r.content,'lxml')
        for x in soup.find('ul', class_='unstyled-list job-list').find_all('li'):
                    row_scrap = []
                    job_title = x.find('h2', attrs={'class' : 'h3-style'}).text.replace('\n', '').strip()
                    job_link = x.find('a')['href']
                    company = "Sanofi"
                    departement = 'N/A'
                    details = x.find('span', attrs={'class' : 'job-location'}).text

                    row_scrap.append(job_title)
                    row_scrap.append(job_link)
                    row_scrap.append(company)
                    row_scrap.append(departement)
                    row_scrap.append(details)

                    sanofi_db.append(row_scrap)

        i = i + 1

    sanofi_db = pd.DataFrame(sanofi_db)
    sanofi_db = sanofi_db.rename(columns={0 : "Position", 1 : "link", 2 : "Company", 3 : "Department", 4 : "Location"}) #
    sanofi_db['desc'] = ''

    i = 0
    while i <= len(sanofi_db)-1:
        r = requests.get('https://fr.jobs.sanofi.com/' + sanofi_db['link'][i])
        soup = BeautifulSoup(r.content,'lxml')
        try :
            sanofi_db['desc'][i] = soup.find('div', attrs={'class' : 'ats-description'}).text
        except :
            sanofi_db['desc'][i] = ''
        i = i + 1
    #df = pd.concat([df, sanofi_db], axis=0)
    print("Sanofi Scrap Finished")

# Scrap Hermes
def scrapHermes():
    r = requests.get('https://talents.hermes.com/fr/')
    soup = BeautifulSoup(r.content,'lxml')

    hermes_db = []

    for x in soup.find_all('div', class_='card-block'):
                    row_scrap = []
                    job_title = x.find('h2', attrs={'class' : 'offer-description bd-highlight block-with-text'}).text.replace('\n', '').strip()
                    job_link = x.find('a')['href']
                    company = "Hermes"
                    departement = 'N/A'
                    details = []
                    for v in x.find('ul', attrs={'class' : 'offer-list_info'}).find_all('li'): 
                        item = v.text
                        details.append(item)

                    #details.append(details_item)

                    row_scrap.append(job_title)
                    row_scrap.append(job_link)
                    row_scrap.append(company)
                    row_scrap.append(departement)
                    row_scrap.append(details)

                    hermes_db.append(row_scrap)

    hermes_db = pd.DataFrame(hermes_db)
    hermes_db = hermes_db.rename(columns={0 : "Position", 1 : "link", 2 : "Company", 3 : "Department", 4 : "Location"}) #
    hermes_db['desc'] = ''

    i = 0
    while i <= len(hermes_db)-1:
        r = requests.get('https://talents.hermes.com/' + hermes_db['link'][i])
        soup = BeautifulSoup(r.content,'lxml')
        try :
            hermes_db['desc'][i] = soup.find('div', attrs={'id' : 'detailOffer'}).text
        except :
            hermes_db['desc'][i] = ''
        i = i + 1
    #df = pd.concat([df, hermes_db], axis=0)
    print("Hermes Scrap Finished")

def scrapFramatome():
    framatome_db = []

    i = 1
    while i <= 44:
        r = requests.get(f'https://framatome-career.talent-soft.com/offre-de-emploi/liste-offres.aspx?page={i}&LCID=1033')
        soup = BeautifulSoup(r.content,'lxml')

        for x in soup.find_all('div', class_='ts-offer-card Layer'):
                        row_scrap = []
                        job_title = x.find('h3', attrs={'class' : 'ts-offer-card__title'}).text.replace('\n', '').strip()
                        job_link = x.find('h3', attrs={'class' : 'ts-offer-card__title'}).find('a')['href']
                        company = "Framatome"
                        departement = 'N/A'
                        details = []
                        for v in x.find('ul', attrs={'class' : 'ts-offer-card-content__list'}).find_all('li'): 
                            item = v.text
                            details.append(item)

                        row_scrap.append(job_title)
                        row_scrap.append(job_link)
                        row_scrap.append(company)
                        row_scrap.append(departement)
                        row_scrap.append(details)

                        framatome_db.append(row_scrap)
        i = i + 1

    framatome_db = pd.DataFrame(framatome_db)
    framatome_db = framatome_db.rename(columns={0 : "Position", 1 : "link", 2 : "Company", 3 : "Department", 4 : "Location"}) #
    framatome_db['desc'] = ''

    i = 0
    while i <= len(framatome_db)-1:
        r = requests.get('https://framatome-career.talent-soft.com/' + framatome_db['link'][i])
        soup = BeautifulSoup(r.content,'lxml')
        try :
            framatome_db['desc'][i] = soup.find('div', attrs={'class' : 'ts-offer-page__content'}).text
        except :
            framatome_db['desc'][i] = ''
        i = i + 1
    #df = pd.concat([df, framatome_db], axis=0)
    print("Framatome Scrap Finished")

def scrapEngie():
    engie_db = []

    i = 1
    while i <= 315:
        r = requests.get(f'https://jobs.engie.com/jobs/search/76073636/page{i}')
        soup = BeautifulSoup(r.content,'lxml')
        for x in soup.find_all('div', class_='jlr_right_hldr'):
            row_scrap = []
            job_title = x.find('p').text.replace('\n', '').strip()
            job_link = x.find('a')['href']
            company = "Engie"
            departement = x.find('p', attrs={'class' : 'jlr_company'}).text.replace('\n', '').strip()
            details = []
            for v in x.find_all('p', attrs={'class' : 'jlr_cat_loc'}): 
                item = v.text.replace('\t', '').replace('\n', '')
                details.append(item)

            row_scrap.append(job_title)
            row_scrap.append(job_link)
            row_scrap.append(company)
            row_scrap.append(departement)
            row_scrap.append(details)

            engie_db.append(row_scrap)
        i = i + 1


    engie_db = pd.DataFrame(engie_db)
    engie_db = engie_db.rename(columns={0 : "Position", 1 : "link", 2 : "Company", 3 : "Department", 4 : "Location"}) #
    engie_db['desc'] = ''

    i = 0
    while i <= 10: #len(engie_db)-1:
        r = requests.get(engie_db['link'][i])
        soup = BeautifulSoup(browser.page_source,'lxml')
        try :
            engie_db['desc'][i] = soup.find('div', attrs={'id' : 'description_box'}).text
        except :
            engie_db['desc'][i] = ''
        i = i + 1
    #df = pd.concat([df, engie_db], axis=0)
    print("Engie Scrap Finished")


In [4]:
scrapRichemont()
scrapDassaultAviation()
scrapAirFrance()
scrapSanofi()
scrapHermes()
scrapFramatome()
scrapEngie()

Richemont Scrap Finished
Dassault Aviation Scrap Finished
Air France Scrap Finished
Sanofi Scrap Finished
Hermes Scrap Finished


KeyboardInterrupt: 

In [ ]:
df = pd.concat([richemont_job_db, dassaultAviation_db, airFrance_db,
               sanofi_db, hermes_db, framatome_db, engie_db], axis=0)
df.head(20)

In [ ]:
df.to_csv('backup/backup_2112.csv')

In [ ]:
len(df)

In [ ]:
engie_db

In [ ]:
# ---------------
# Air Liquide Scrap
# ---------------
#    - Issue : redirect to homepage instead of career page

#browser.get('https://www.airliquide.com/fr/carrieres/offres-emploi')
#soup = BeautifulSoup(browser.page_source,'lxml')
#airLiquide_db = []
#for x in soup.find_all('tr'):
#    row_scrap = []
#    job_title = x.find('a').text.replace('\n', '').strip()
#    job_link = x.find('h3', attrs={'class' : 'ts-offer-card__title'}).find('a')['href']
#    company = "Framatome"
#    departement = 'N/A'
#    details = []
#    for v in x.find('ul', attrs={'class' : 'ts-offer-card-content__list'}).find_all('li'): 
#        item = v.text
#        details.append(item)

#    row_scrap.append(job_title)
#    row_scrap.append(job_link)
#    row_scrap.append(company)
#    row_scrap.append(departement)
#    row_scrap.append(details)

#    airLiquide_db.append(row_scrap)

#airLiquide_db = pd.DataFrame(airLiquide_db)
#airLiquide_db = airLiquide_db.rename(columns={0 : "Position", 1 : "link", 2 : "Company", 3 : "Department", 4 : "Location"}) #
#airLiquide_db

In [ ]:
## https://www.airliquide.com/fr/carrieres/offres-emploi
https://jobsearch.alstom.com/search/?createNewAlert=false&q=&locationsearch=
https://cc.wd3.myworkdayjobs.com/fr-FR/ChanelCareers
https://jobs.danone.com/search/?createNewAlert=false&q=&locationsearch=&optionsFacetsDD_country=FR&optionsFacetsDD_facility=&optionsFacetsDD_department=Experienced+professionals
https://careers.3ds.com/jobs?woc=%7B%22country%22%3A%5B%22country%2Ffrance%22%5D%7D&wocset=6
http://careers.disneylandparis.com/en/management-business/supply-chain-procurement
https://www.edf.fr/edf-recrute
https://jobs.engie.com
https://recrutement.fnacdarty.com/accueil.aspx?LCID=1036
https://www.place-emploi-public.gouv.fr
https://jobs.gecareers.com/global/en/search-results
https://www.invivo-group.com/fr/nos-offres
https://kering.wd3.myworkdayjobs.com/fr-FR/Kering?source=LinkedIn_Slots
https://careers.loreal.com/en_US/jobs/SearchJobs/
https://www.lisi-aerospace.com/en/join-us/careers/
https://www.lvmh.fr/talents/nous-rejoindre/nos-offres/liste-des-offres/?job=&place=&experience=&activity=&contract=&reference=#gt_offers-results
https://www.mbda-systems.com/jobs/?gestmax%5Bvac_sector%5D=&gestmax%5Bvac_localisation%5D=001&gestmax%5Bvac_job_type%5D=
https://jobs.moncler.com/search/?createNewAlert=false&q=&locationsearch=
https://motul-recrute.talent-soft.com/job/list-of-jobs.aspx
https://www.naval-group.com/fr/nous-rejoindre-85?keywords=&offerFamilyCategory=&contractType=&country=&city=&op=Rechercher&form_build_id=form-NzrJ7AsvKwvEKPwLDA2P5Qu6e3T6EKUpkTDoHGZS6Is&form_id=talent_soft_offers_filters_form#offer-list-content
https://nexter-recrutement.profils.org/accueil.aspx?LCID=1036
https://orange.jobs/jobs/search.do?keyword=
https://pernodricard.wd3.myworkdayjobs.com/fr-FR/pernod-ricard
https://jobs.groupe-psa.com/offre-de-emploi/liste-offres.aspx?mode=layer&lcid=1036&facet_JobDescription_Contract=577
https://renault.referrals.selectminds.com/
https://www.safran-group.com/jobs
https://joinus.saint-gobain.com/fr
https://www.sodern.com/website/fr/ref/Carrières_262.html
https://hris-suez.csod.com/ats/careersite/search.aspx?site=8&c=hris-suez&sid=%5e%5e%5eHJe5gko1mldbDMyZ8oI9Lw%3d%3d
https://careers.hr.technipfmc.com
https://emploi.thalesgroup.com/recherche-d%27offres
https://krb-sjobs.brassring.com/TGnewUI/Search/Home/Home?partnerid=30080&siteid=6559#home
https://career012.successfactors.eu/career?company=VALLOUREC&site=VjItcmY2YVFFcnJMYWhIb3RmMzhTYU9Ldz09
https://emplois.vinci.com/recherche-d%27offres
https://www.nestle.fr/jobs/search-jobs?keyword=&country=FR&location=&career_area=All&company=All
https://www.smcp.com/fr/talents/offres-d-emploi/?keywords=&geographicalLocation=22&offerCountry=79&offerRegion=&organisation=&offerFamilyCategory=&contractType=&experienceLevel=
https://pfizer.wd1.myworkdayjobs.com/PfizerCareers/5/refreshFacet/318c8bb6f553100021d223d9780d30be
https://careers.faurecia.com/search/?createNewAlert=false&q=&locationsearch=france&optionsFacetsDD_customfield3=&optionsFacetsDD_country=&optionsFacetsDD_shifttype=Unlimited
https://www.emploi.sncf.com/nos-offres/contrat/577-578/localisation/40629/
https://careers.ratpdev.com/offre-de-emploi/liste-offres.aspx?page=3&LCID=1036
https://arianegroup.wd3.myworkdayjobs.com/EXTERNALALL